In [1]:
# This works. Processes videos in np arrays, which we can save to folder or use directly

import cv2, os
print(cv2.__version__)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

from IPython.core.display import Image, display

import numpy as np

3.4.2


In [2]:
# Dowload youtube-dl

!curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!chmod a+rx /usr/local/bin/youtube-dl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     3    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     3    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   599    0   599    0     0    599      0 --:--:--  0:00:01 --:--:--   599
100 1655k  100 1655k    0     0   827k      0  0:00:02  0:00:02 --:--:-- 2918k


In [3]:
# download clip
!youtube-dl https://www.youtube.com/watch?v=bR4Gq9qfpnM

[youtube] bR4Gq9qfpnM: Downloading webpage
[youtube] bR4Gq9qfpnM: Downloading video info webpage
[download] Destination: Drone racing - First Person View (FPV)-bR4Gq9qfpnM.mp4
[download] 100% of 24.48MiB in 00:00


In [0]:
ls drone/train

In [0]:
#os.mkdir("video_frames/train")
os.mkdir("drone")
os.mkdir("drone/train")
#! rm -r drone/train

In [0]:
def format_ix(ix): # Takes in int and returns str padded w 0s. Want to keep files in order. Good up to 99k.
    return ('0000'+str(ix))[-5:]

In [6]:
format_ix(100)

'00100'

In [0]:
vid = "Drone racing - First Person View (FPV)-bR4Gq9qfpnM.mp4"

vidcap = cv2.VideoCapture(vid)
success,image = vidcap.read() # image is np array
count = 0
success = True
while success:
  cv2.imwrite("drone/train/frame"+ format_ix(count) +".jpg", image)     # save frame as JPEG file
  success,image = vidcap.read()
  count += 1

In [0]:
image

In [37]:
#
transform = transforms.Compose([
    transforms.Resize((128)),
    #transforms.RandomCrop(64, pad_if_needed=True),
    transforms.CenterCrop(128),
    transforms.ToTensor(), 
])

# Obnoxious. Images have to be within folder within root
dataset = datasets.ImageFolder(root='./drone', transform=transform)
len(dataset.imgs)

2969

In [0]:
sample = dataset[2968][0].data.cpu()
save_image(sample.data.cpu(), 'sample_image.png')
display(Image('sample_image.png', width=300, unconfined=True))
display(Image('drone/train/frame2968.jpg', width=300, unconfined=True))

In [0]:
# Stitching frames back together into video

img1 = cv2.imread('drone/train/frame100.jpg') # this is a np array
height , width , layers =  img1.shape
fps = 30

video = cv2.VideoWriter('drone.avi', cv2.VideoWriter_fourcc(*"MJPG"), fps, (width,height))

for i in range(count-1):
    img = cv2.imread('drone/train/frame'+str(i)+'.jpg')
    video.write(img)
    
cv2.destroyAllWindows()
video.release()

In [19]:
img1.shape

(720, 1280, 3)

In [0]:
from google.colab import files

files.download('drone.avi')  # from colab to browser download

In [0]:
# Stitching frames back together into video

img1 = cv2.imread('drone/train/frame100.jpg')
img2 = cv2.imread('drone/train/frame101.jpg')
img3 = cv2.imread('drone/train/frame102.jpg')

height , width , layers =  img1.shape

video = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*"MJPG"),1,(width,height))

video.write(img1)
video.write(img2)
video.write(img3)

cv2.destroyAllWindows()
video.release()

writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30,(640,480))
for frame in range(10):
    writer.write(np.random.randint(0, 255, (480,640,3)).astype('uint8'))
writer.release()